In [1]:
from __future__ import print_function
from ipywidgets import Controller, FloatText, HTML, VBox
from pythreejs.install import install
from traitlets.traitlets import link, dlink

# A game controller widget

In [2]:
pad = Controller()
pad

In [4]:
pad.axes[1].value

0.5901197195053101

# A jupyter - threejs bridge

In [5]:
import numpy as np
from pythreejs import *

nx, ny= (20, 20)
xmax = 1
x = np.linspace(-xmax, xmax, nx)
y = np.linspace(-xmax, xmax, ny)
xx, yy = np.meshgrid(x, y)
z = xx**2-yy**2

surf_g = SurfaceGeometry(z=list(z[::-1].flat), 
                          width=2 * xmax,
                          height=2 * xmax,
                          width_segments=nx-1,
                          height_segments=ny-1)

surf = Mesh(geometry=surf_g, material=LambertMaterial(map=height_texture(z[::-1], 'YlGnBu_r')))
surfgrid = SurfaceGrid(geometry=surf_g, material=LineBasicMaterial(color='black'))
hover_point = Mesh(geometry=SphereGeometry(radius=0.05), material=LambertMaterial(color='hotpink'))
scene = Scene(children=[surf, surfgrid, hover_point, AmbientLight(color='#777777')])
c = PerspectiveCamera(position=[0,3,3], up=[0,0,1], 
                      children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.6)])
click_picker = Picker(root=surf, event='dblclick')
hover_picker = Picker(root=surf, event='mousemove')
renderer = Renderer(camera=c, scene = scene, controls=[OrbitControls(controlling=c), click_picker, hover_picker])

def f(change):
    value = change['new']
    print('Clicked on %s' % value)
    point = Mesh(geometry=SphereGeometry(radius=0.05), 
                 material=LambertMaterial(color='red'),
                 position=value)
    scene.children = list(scene.children) + [point]

click_picker.observe(f, names='point')

link((hover_point, 'position'), (hover_picker, 'point'))

h = HTML()
def g(change):
    h.value="Green point at (%.3f, %.3f, %.3f)" % tuple(change['new'] )
    
hover_picker.observe(g, names='point')
VBox([h, renderer])

# Terrain modeling? Let us check out the Grand Canyon

In [6]:
from pythreejs import *
import numpy as np
import gdal as gd

In [7]:
gc_ds = gd.Open('gc_dem.tif')
dem = gc_ds.ReadAsArray()[::20, ::20]
gt = gc_ds.GetGeoTransform()

z = (dem - np.mean(dem)) / 1000
nx, ny = z.shape

surf_g = SurfaceGeometry(z=list(z.flat), height_segments=nx - 1, width_segments=ny - 1)
surf = Mesh(geometry=surf_g, material=LambertMaterial(map=height_texture(z, colormap='terrain')), scale=(10, 10, 1))
scene = Scene(children=[AmbientLight(color='#777777'),
                        surf, 
                        DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)])

In [8]:
c = PerspectiveCamera(position=[0, 10, 10], up=[0, 0, 1], 
                      children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)],
                      aspect=2)
width = 950
height = 950 / c.aspect
c.look_at(c.position, (1, 0, 0))
fly_controls = FlyControls(controlling=c)
renderer = Renderer(camera=c, scene=scene, width=str(width), height=str(height), controls=[fly_controls])

In [9]:
c.position = [0, 10, 10]
c.look_at(c.position, (1, 0, 0))

In [10]:
renderer

# Control the camera position with the gamepad

In [11]:
factor = 10
def affine(constant, factor):
    return lambda x: constant + factor * x

pad.links = []

def setup():
    if pad.connected:
        pad.links.append(dlink((pad.axes[1], 'value'), (fly_controls, 'pitch'), affine(0.0, factor)))
        pad.links.append(dlink((pad.axes[0], 'value'), (fly_controls, 'roll'), affine(0.0, -factor)))
        pad.links.append(dlink((pad.axes[3], 'value'), (fly_controls, 'forward_speed'), affine(0.0, 2 * factor)))
        pad.links.append(dlink((pad.axes[2], 'value'), (fly_controls, 'yaw'), affine(0.0, factor)))
        pad.links.append(dlink((pad.buttons[5], 'value'), (surf, 'scale'), lambda x: (10, 10, 1 - x)))
    if not pad.connected:
        for l in pad.links:
            l.unlink()
        pad.links = []

pad.observe(setup, names=['connected'])
setup()